In [1]:
# require at least Python 3.5 for async/await to work
import sys
ver=sys.version_info
assert (ver[0]>=3 and ver[1]>=5)
sys.path.append('../../quant_container/src')
sys.path.append('../../quant-container/src')
sys.path.append('../src')
%load_ext autoreload
%autoreload 2
from mosaicsmartdata.core.fx import FXPricingContextGenerator
from mosaicsmartdata.common import qc_csv_helper
from mosaicsmartdata.core.quote import Quote
import csv, copy
import pickle, cloudpickle

In [166]:
try:
    with open('data.pickle', 'rb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        quote_dict = pickle.load(f)
except:
#if True:
    file_path ='../resources/fx/'
    filenames = ['OIS-N156898469.csv','EUR_JPY_GBP_till_SN.csv','EUR_JPY_GBP_from_SW_till_1Y.csv']
    quote_dict_0 = qc_csv_helper.get_ric_quotes(file_path+filenames[0])
    print(len(quote_dict_0), quote_dict_0.keys())
    quote_dict_1 = qc_csv_helper.get_ric_quotes(file_path+filenames[1])
    print(len(quote_dict_1), quote_dict_1.keys())
    quote_dict_2 = qc_csv_helper.get_ric_quotes(file_path+filenames[2])
    print(len(quote_dict_2), quote_dict_2.keys())
    quote_dict = dict(quote_dict_0,**quote_dict_1)
    quote_dict = dict(quote_dict,**quote_dict_2)
    with open('data.pickle', 'wb') as f:
        pickle.dump(quote_dict, f)

In [2]:
from aiostreams import run
import aiostreams.operators as op
from mosaicsmartdata.core.markout import MarkoutCalculator
import logging

In [170]:
# extract just the first quote from every stream
tiny_quote_dict = {key:[quotes[0]] for key, quotes in quote_dict.items()}
print([(key,quotes[0].mid, quotes[0].instrument.ccy, quotes[0].instrument.tenor) for key, quotes in tiny_quote_dict.items()])
with open('../resources/fx/data.pickle', 'wb') as f:
    pickle.dump(tiny_quote_dict, f)


[('JPY=', 111.24000000000001, ('USD', 'JPY'), 'SPOT'), ('GBP=', 1.2744499999999999, ('GBP', 'USD'), 'SPOT'), ('EUR1M=', 19.21, ('EUR', 'USD'), '1M'), ('USD6MOIS=', 1.185, 'USD', '6M'), ('EURSN=', 0.546, ('EUR', 'USD'), 'SN'), ('EUR1Y=', 231.8, ('EUR', 'USD'), '1Y'), ('EUR6M=', 111.515, ('EUR', 'USD'), '6M'), ('GBP3M=', 36.41, ('GBP', 'USD'), '3M'), ('JPY1M=', -16.13, ('USD', 'JPY'), '1M'), ('GBPSW=', 4.62, ('GBP', 'USD'), 'SW'), ('GBP1M=', 13.84, ('GBP', 'USD'), '1M'), ('USD3MOIS=', 1.158, 'USD', '3M'), ('USDSWOIS=', 1.1225, 'USD', 'SW'), ('GBPTN=', 0.405, ('GBP', 'USD'), 'TN'), ('USD1MOIS=', 1.151, 'USD', '1M'), ('USD1YOIS=', 1.2505, 'USD', '1Y'), ('EUR=', 1.11995, ('EUR', 'USD'), 'SPOT'), ('JPY3M=', -45.32, ('USD', 'JPY'), '3M'), ('GBPON=', 0.392, ('GBP', 'USD'), 'ON'), ('JPYON=', -0.4695, ('USD', 'JPY'), 'ON'), ('JPY6M=', -92.8, ('USD', 'JPY'), '6M'), ('GBP6M=', 70.87, ('GBP', 'USD'), '6M'), ('EURON=', 0.54, ('EUR', 'USD'), 'ON'), ('JPYSN=', -0.4495, ('USD', 'JPY'), 'SN'), ('JPY1Y='

In [ ]:
# import datetime
# a = datetime.date(2107,1,1)
# b = datetime.date(2107,1,2)
# from mosaicsmartdata.common.quantlib.bond.fixed_bond import pydate_to_qldate
# pydate_to_qldate(b)< pydate_to_qldate(a)


In [135]:
from datetime import timedelta
from random import shuffle

with open('../resources/fx/data.pickle', 'rb') as f:
        tiny_quote_dict = pickle.load(f)

# that one quote happens to have the wrong date
tiny_quote_dict['USDSWOIS='][0].timestamp -= timedelta(days=1)
tiny_quote_dict['USDSWOIS='][0].instrument.spot_settle_date -= timedelta(days=1)
tiny_quote_dict['USDSWOIS='][0].instrument.maturity_date -= timedelta(days=1)

tiny_quotes_list =[quotes for key, quotes in tiny_quote_dict.items()]
#print(tiny_quotes_list)
gbpsn = [q[0] for q in tiny_quotes_list if q[0].sym == 'GBPSN='][0]
tiny_quotes_list += [[gbpsn]]
#pc.extra
#print([q[0].sym for q in tiny_quotes_list])

In [178]:
#print(tiny_quotes_list)
my_quotes = [q[0] for q in tiny_quotes_list]

#shuffle(my_quotes)

#[print(q[0].instrument.sym, q[0].timestamp.date()) for q in tiny_quotes_list]
# stream = op.merge_sorted(tiny_quotes_list, lambda x: x.timestamp) |\
#             op.flat_map(FXPricingContextGenerator()) > []

stream = my_quotes |  op.flat_map(FXPricingContextGenerator()) > []
    
run(stream)
pc = stream.sink[-1]
    
for quote in tiny_quotes_list:
    #if 'OIS' not in sym: #and 'TN' not in sym:
            mid = quote[0].mid
            instr = quote[0].instrument
            est = instr.price(pc)
            if abs(mid-est) > 1e-6:
                print(quote[0].sym, mid, est, abs(mid-est), 
                      quote[0].timestamp.date(), 
                      instr.settle_date, 
                      instr.maturity_date)

GBPSN= 0.40700000000000003 0.35467990074 0.0523200992597 2017-06-26 2017-06-28 2017-06-29
GBPON= 0.392 0.398493440044 0.00649344004361 2017-06-26 2017-06-26 2017-06-27
JPYON= -0.4695 -0.45975891168 0.00974108832016 2017-06-26 2017-06-26 2017-06-27
EURON= 0.54 0.542486475568 0.00248647556765 2017-06-26 2017-06-26 2017-06-27
GBPSN= 0.40700000000000003 0.35467990074 0.0523200992597 2017-06-26 2017-06-28 2017-06-29


In [188]:
from mosaicsmartdata.core.curve_utils import construct_OIS_curve, discounting_factor, get_rate
import datetime

syms = ['GBPON=', 'GBPTN=', 'GBPSN=']
min_quotes = []
for sym in syms:
    min_quotes.append([q[0] for q in tiny_quotes_list if q[0].sym == sym][0])
print([q.sym for q in min_quotes])
    
ontn = min_quotes[0].mid + min_quotes[1].mid
print(ontn,pc.fair_fwd_points_extended(instr.ccy, min_quotes[0].instrument.settle_date, min_quotes[1].instrument.maturity_date) )
curvedata = pc.curve['GBP'].source_data
print([q for q in curvedata if q[3] == 'ONTN'])

for q in min_quotes:
    #print(q)
    instr =q.instrument 
    a1 = instr.settle_date
    a2 = instr.maturity_date

    my_curve = pc.curve['USD']
    print(instr.sym)
    # both these commands run fine
    if q.sym == 'GBPSN=':
        print(get_rate(my_curve, a1, a2), discounting_factor(my_curve, a1, a2))
    print(pc.fair_fwd_points_extended(instr.ccy, a1, a2))

# # I'm constructing another curve, nothing to do with ours! 
# ois_curve = construct_OIS_curve(my_curve.source_data)
# # and the very same command now returns a different value, 1.1178 instead of 1.1163!
# print(get_rate(my_curve, a1, a2), discounting_factor(my_curve, a1, a2))
# # and this command now throws an error!
# print(pc.fair_fwd_points_extended(instr.ccy, a1, a2))


['GBPON=', 'GBPTN=', 'GBPSN=']
0.797 (1.2743696506560092, 1.2744499999999999, 0.99993695371023528, 1.0)
[(datetime.date(2017, 6, 26), datetime.date(2017, 6, 28), -0.01125697249402657, 'ONTN', 1.0000625406916654)]
GBPON=
((1.2744095000000135, 1.2744499999999999, 0.99996822158579279, 1.0), (1.2743696506560092, 1.2744499999999999, 0.99993695371023528, 1.0))
GBPTN=
(1.2744095000000135, 1.2744499999999999, 0.99996822158579279, 1.0)
GBPSN=
1.1163659523685254 0.996903786651
(1.2744854679900739, 1.2744499999999999, 0.99693153052002792, 0.99690378665121415)


In [146]:
with open('OIS_curve.pickle', 'rb') as f:
        OIS_df = pickle.load(f)
print(OIS_df)
        
ois_curve = construct_OIS_curve(OIS_df)

print(get_rate(other_curve, a1, b), discounting_factor(other_curve, a1, b))
print(pc.fair_fwd_points_extended(instr.ccy, instr.settle_date, instr.maturity_date))


[(datetime.date(2017, 6, 28), datetime.date(2017, 7, 5), 1.1225, 'SW'), (datetime.date(2017, 6, 28), datetime.date(2017, 7, 28), 1.151, '1M'), (datetime.date(2017, 6, 28), datetime.date(2017, 9, 28), 1.158, '3M'), (datetime.date(2017, 6, 28), datetime.date(2017, 12, 28), 1.185, '6M'), (datetime.date(2017, 6, 28), datetime.date(2018, 6, 28), 1.2505, '1Y')]
1.1178438188697015
1.1178438188697015 0.996899694185


RuntimeError: more than one instrument with pillar June 29th, 2017

In [ ]:
from mosaicsmartdata.core.curve_utils import construct_OIS_curve, discounting_factor
import datetime

with open('EUR_curve.pickle', 'rb') as f:
        EUR_df = pickle.load(f)
print(EUR_df)
        
eur_curve =  construct_OIS_curve(EUR_df)
a1 = datetime.date(2017, 6, 26)
a2 = datetime.date(2017, 6, 27)
b = datetime.date(2017, 6, 28)
df1 = discounting_factor(eur_curve, a1, b)
df2 = discounting_factor(eur_curve, a2, b)
print(df1, df2)

In [66]:
pc.extra['EUR']
for sym, quote in tiny_quote_dict.items():
    if 'EUR' in sym:
        print(quote[0].mid, quote[0].instrument.tenor, quote[0].instrument.sym)
print(pc.extra['GBP'])

19.21 1M EUR1M=
0.546 SN EURSN=
231.8 1Y EUR1Y=
111.515 6M EUR6M=
0.54 ON EURON=
1.11995 SPOT EUR=
0.5449999999999999 TN EURTN=
5.9399999999999995 SW EURSW=
55.269999999999996 3M EUR3M=
{datetime.date(2017, 7, 5): (0.7843678622524536, 'SW'), datetime.date(2017, 12, 28): (0.780313014762742, '6M'), datetime.date(2017, 9, 28): (0.7824169014569385, '3M'), datetime.date(2017, 6, 26): (0.7847012756025467, 'ONTN'), datetime.date(2017, 7, 28): (0.7838010274063868, '1M'), datetime.date(2017, 6, 27): (0.7846771387062009, 'TN'), datetime.date(2017, 6, 29): (0.7846271455727375, 'SN'), datetime.date(2018, 6, 28): (0.7759384587689582, '1Y')}
